# Density-matrix simulation

This notebook shows how to get the density matrix corresponding to a noisy circuit.

We first define a quantum circuit:

In [ ]:
from qat.lang.AQASM import Program, H
from qat.lang.AQASM.qftarith import QFT
prog = Program()
reg = prog.qalloc(4)
prog.apply(H, reg[1])
prog.apply(QFT(4), reg)
qft_circuit = prog.to_circ(inline=True) #convert program to circuit
%qatdisplay qft_circuit

We then create a hardware model with perfect gates and T1/T2 noise:

In [ ]:
from qat.hardware import DefaultGatesSpecification, HardwareModel
from qat.quops import ParametricPureDephasing, ParametricAmplitudeDamping

ibm_gates_spec = DefaultGatesSpecification(gate_times={"H":60, "X":120, "Y":120, "S":1,"T":1,
                                                       'D-T':1, "Z":1, "RZ":lambda angle : 1,
                                                       "PH": lambda angle : 1, "CNOT":386,
                                                       "SWAP":150, "C-PH":lambda angle :150, "C-T":150, "C-S":150})
T1, T2 = 44000, 38900 #nanosecs
hardware_model = HardwareModel(ibm_gates_spec, 
                               idle_noise=[ParametricAmplitudeDamping(T_1=T1),
                                           ParametricPureDephasing(T_phi=1/(1/T2 - 1/(2*T1)))])

%qatdisplay qft_circuit -hw hardware_model

Finally, we compute the density matrix and check that the final state is mixed, i.e $\mathrm{Tr}(\rho^2) < \mathrm{Tr}(\rho) = 1$

Note: if no hardware model is specified, we assume ideal gates.

In [ ]:
from qat.qpus import NoisyQProc
from qat.noisy import compute_density_matrix
qpu = NoisyQProc(hardware_model=hardware_model)
rho = compute_density_matrix(qft_circuit, qpu)

import numpy as np
print("Tr(rho) = ", np.trace(rho).real)
print("Tr(rho^2) = ", np.trace(np.dot(rho, rho)).real)

In [ ]:
from qat.noisy.plot import plot_density_matrix
%matplotlib inline
plot_density_matrix(rho)

Note: because of the high memory overhead of storing the density matrix, this method is limited to a small number of qbits.